# Compare acquisition optimizers on precision and speed

In [ ]:
from gaussian_process import GaussianProcess, GPfunctions
from gaussian_process.kernels import Matern2_5Kernel
from acquisition import AcquisitionFunction, LowerConfidenceBound
from acquisition.optimization import (
    GradientBinarySearchAcquisitionOptimizer,
    DIRECT_LBFGSB_AcquisitionOptimizer,
    DE_LBFGSB_AcquisitionOptimizer,
    SHGO_LBFGSB_AcquisitionOptimizer,
    DA_LBFGSV_AcquisitionOptimizer,
)

import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
X = np.linspace(start=0.0, stop=1.0, num=1_000)

x_known = [0.0, 1.0, 0.36492029, 0.10920915, 0.02193941, 0.00234551]
f_known = [
    0.00000000e00,
    1.00000000e00,
    1.32713151e-01,
    1.17465087e-02,
    4.50827643e-04,
    3.76193533e-06,
]
g_known = [-0.00200667, 2.00197593, 0.72928723, 0.21684655, 0.04195951, 0.00269369]

x_known = np.array(x_known)
f_known = np.array(f_known)
g_known = np.array(g_known)

gp = GaussianProcess(Matern2_5Kernel(), x_known, f_known, g_known)

acquisition = LowerConfidenceBound(gp)

fig, (ax1, ax2) = plt.subplots(2, 1)
GPfunctions.plot_gp(ax1, X, gp.mean(X), gp.std_deviation(X))
ax2.plot(X, acquisition(X), label="Acquisition")
GPfunctions.plot_label(ax1, "Gaussian Process")
GPfunctions.plot_label(ax2, "Acquisition")
plt.show()

print(f"Max acquisition at roughly at {X[np.argmax(acquisition(X))]}")

In [ ]:
class CountedAcquisition:
    def __init__(self, acquisition: AcquisitionFunction) -> None:
        self.acquisition = acquisition
        self.f_count = 0
        self.g_count = 0
        self.fg_count = 0

    def evaluate(self, x: np.ndarray) -> np.ndarray:
        self.f_count += 1
        return self.acquisition.evaluate(x)

    def __call__(self, x: np.ndarray) -> np.ndarray:
        return self.evaluate(x)

    def derivative(self, x: np.ndarray) -> np.ndarray:
        self.g_count += 1
        return self.acquisition.derivative(x)

    def value_derivative(self, x: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        self.fg_count += 1
        return self.acquisition.value_derivative(x)

    def __str__(self) -> str:
        return f"f:{self.f_count}, g:{self.g_count}, fg:{self.fg_count}"

In [ ]:
optimizers = [
    GradientBinarySearchAcquisitionOptimizer(),
    DIRECT_LBFGSB_AcquisitionOptimizer(),
    DE_LBFGSB_AcquisitionOptimizer(),
    SHGO_LBFGSB_AcquisitionOptimizer(),
    DA_LBFGSV_AcquisitionOptimizer(),
]

for optimizer in optimizers:
    counted_acquisition = CountedAcquisition(acquisition)
    start = time.time()
    x_best = optimizer.maximize(counted_acquisition, 0.0, 1.0, x_known)
    for _ in range(100 - 1):
        assert x_best == optimizer.maximize(counted_acquisition, 0.0, 1.0, x_known)
    end = time.time()
    print(
        f"{optimizer.__class__.__name__} Calls:({counted_acquisition}), x_best:{x_best}, f{acquisition(x_best)}:, time:{end-start}"
    )